In [41]:
from sfepy.discrete.fem import Mesh, FEDomain, Field
from sfepy.discrete import Function
import numpy as np

In [42]:
mesh = Mesh.from_file('part1.mesh')
# print(mesh.cmesh_tdim[2])

sfepy: reading mesh (part1.mesh)...
sfepy:   number of vertices: 33
sfepy:   number of cells:
sfepy:     2_3: 33
sfepy: ...done in 0.00 s


In [45]:
domain = FEDomain('domain', mesh)
# domain.surface_groups
# print(domain.geom_els['2_3'])
tags = [1, 2]
def get_edge(coords, domain = None):
   coord0 = coords[tags[0] - 1]
   coord1 = coords[tags[1] - 1]
   x1, y1 = coord1[0] - coord0[0], coord1[1] - coord0[1]
   x2, y2 = coords[:, 0] - coord0[0], coords[:, 1] - coord0[1]
   print(np.where(np.abs(x1 * y2 - x2 * y1) < 1e-14)[0])
   return np.where(np.abs(x1 * y2 - x2 * y1) < 1e-14)[0]

functions = Function('get_edge', get_edge)
constraints = domain.create_region('Constraints', "vertices by get_edge", 'facet', functions={'get_edge' : Function('get_edge', get_edge)})
print(constraints)

[0 1 6 7 8]
Region:Constraints
  can:
    tuple: (True, True, False)
  can_cells:
    False
  can_edges:
    True
  can_faces:
    False
  can_vertices:
    True
  cmesh:
    CMesh: n_coor: 33, dim 2, tdim: 2, n_el 33
  definition:
    vertices by get_edge
  dim:
    2
  domain:
    FEDomain:domain
  entities:
    list: [array([0, 1, 6, 7, 8], dtype=uint32), array([ 3,  6, 14, 45], dtype=uint32), array([], dtype=uint32)]
  extra_options:
    None
  is_empty:
    False
  kind:
    facet
  kind_tdim:
    1
  mirror_maps:
    dict with keys: []
  mirror_regions:
    dict with keys: []
  n_v_max:
    33
  name:
    Constraints
  parent:
    None
  parse_def:
    E_VBF<vertices by get_edge>
  shape:
    Struct
  tdim:
    2
  true_kind:
    edge


In [126]:
forces = domain.create_region('Forces', 'vertex 22', 'vertex')
# constraints = domain.create_region('Constraints', '(vertex 9 +v vertex 7)', 'facet')
constraints = domain.create_region('Constraints', '(vertex 7 +v vertex 8)  +e (vertex 21 +v vertex 22) +e (vertex 22 +v vertex 23)', 'facet')
# domain.create_region('Constraints', '(vertices of group 18)', 'facet')
print(domain.regions)

[
  Region:Forces
    can:
      tuple: (True, False, False)
    can_cells:
      False
    can_edges:
      False
    can_faces:
      False
    can_vertices:
      True
    cmesh:
      CMesh: n_coor: 142, dim 2, tdim: 2, n_el 210
    definition:
      vertex 22
    dim:
      2
    domain:
      FEDomain:domain
    entities:
      list: [array([22], dtype=uint32), array([], dtype=uint32), array([], dtype=uint32)]
    extra_options:
      None
    is_empty:
      False
    kind:
      vertex
    kind_tdim:
      0
    mirror_maps:
      dict with keys: []
    mirror_regions:
      dict with keys: []
    n_v_max:
      142
    name:
      Forces
    parent:
      None
    parse_def:
      E_VI<vertex 22>
    shape:
      Struct
    tdim:
      2
    true_kind:
      vertex
  Region:Constraints
    can:
      tuple: (True, True, False)
    can_cells:
      False
    can_edges:
      True
    can_faces:
      False
    can_vertices:
      True
    cmesh:
      CMesh: n_coor: 142, dim 2,

In [112]:
min_x, max_x = domain.get_mesh_bounding_box()[:, 0]
print(min_x, max_x)

0.0 1.0


In [113]:
eps = 1e-8 * (max_x - min_x)
omega = domain.create_region('Omega', 'all')

In [114]:
field = Field.from_args('fu', np.float64, 'vector', omega, approx_order=2)

In [115]:
from sfepy.discrete import (FieldVariable, Material, Integral, Function, Equation, Equations, Problem)
u = FieldVariable('u', 'unknown', field)
v = FieldVariable('v', 'test', field, primary_var_name='u')


In [116]:
from sfepy.mechanics.matcoefs import stiffness_from_lame
m = Material('m', D=stiffness_from_lame(dim=2, lam=1.0, mu=1.0))
f = Material('f', val=[[0.02], [0.01]])


In [117]:
integral = Integral('i', order=3)

In [118]:
from sfepy.terms import Term
t1 = Term.new('dw_lin_elastic(m.D, v, u)',
              integral, omega, m=m, v=v, u=u)
t2 = Term.new('dw_volume_lvf(f.val, v)',
              integral, omega, f=f, v=v)
eq = Equation('balance', t1 + t2)
eqs = Equations([eq])


In [119]:
from sfepy.discrete.conditions import Conditions, EssentialBC
fix_u = EssentialBC('fix_u', constraints, {'u.all' : 0.0})
def shift_u_fun(ts, coors, bc=None, problem=None, shift=0.0):
                val = shift
                return val
bc_fun = Function('shift_u_fun', shift_u_fun,
                  extra_args={'shift' : 0.1})
shift_u = EssentialBC('shift_u', forces, {'u.0' : bc_fun})

In [120]:
from sfepy.base.base import IndexedStruct
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton
ls = ScipyDirect({})
nls_status = IndexedStruct()
nls = Newton({}, lin_solver=ls, status=nls_status)


In [121]:
pb = Problem('elasticity', equations=eqs)
pb.save_regions_as_groups('regions')


sfepy: saving regions as groups...
sfepy:   Omega
sfepy:   Forces
sfepy:   Constraints
sfepy: ...done


In [122]:
pb.set_bcs(ebcs=Conditions([fix_u, shift_u]))
pb.set_solver(nls)
status = IndexedStruct()
variables = pb.solve(status=status)
print('Nonlinear solver status:\n', nls_status)
print('Stationary solver status:\n', status)
pb.save_state('linear_elasticity.vtk', variables)


sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.00 s
sfepy: matrix shape: (985, 985)
sfepy: assembling matrix graph...
sfepy: ...done in 0.00 s
sfepy: matrix structural nonzeros: 20249 (2.09e+00% fill)
sfepy: updating variables...
sfepy: ...done
sfepy: updating materials...
sfepy:     f
sfepy:     m
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 3.619253e-01 (rel: 1.000000e+00)
sfepy:   residual:    0.00 [s]
sfepy:     matrix:    0.00 [s]
sfepy:      solve:    0.01 [s]
sfepy: nls: iter: 1, residual: 2.875330e-14 (rel: 7.944540e-14)
sfepy: solved in 1 steps in 0.02 seconds
Nonlinear solver status:
 IndexedStruct
  condition:
    0
  err:
    2.8753296912172944e-14
  err0:
    0.3619252552108743
  ls_n_iter:
    -1
  n_iter:
    1
  time_stats:
    dict with keys: ['residual', 'matrix', 'solve']
Stationary solver status:
 IndexedStruct
  n_step:
    1
  time:
    0.02204818999962299


In [123]:
# #!/usr/bin/env python
# """
# Print various information about a mesh.
# """
# import sys
# sys.path.append('.')
# from argparse import RawDescriptionHelpFormatter, ArgumentParser

# import numpy as nm

# from sfepy.base.base import output
# from sfepy.discrete.fem import Mesh, FEDomain
# from sfepy.discrete.common.extmods.cmesh import graph_components

# def show_mesh_info(filename):
#     mesh = Mesh.from_file(filename)

#     output(mesh.cmesh)
#     output('element types:', mesh.descs)
#     output('nodal BCs:', sorted(mesh.nodal_bcs.keys()))

#     bbox = mesh.get_bounding_box()
#     output('bounding box:\n%s'
#            % '\n'.join('%s: [%14.7e, %14.7e]' % (name, bbox[0, ii], bbox[1, ii])
#                        for ii, name in enumerate('xyz'[:mesh.dim])))

#     output('centre:           [%s]'
#            % ', '.join('%14.7e' % ii for ii in 0.5 * (bbox[0] + bbox[1])))
#     output('coordinates mean: [%s]'
#            % ', '.join('%14.7e' % ii for ii in mesh.coors.mean(0)))

# #     if not options.detailed: return

#     domain = FEDomain(mesh.name, mesh)

#     for dim in range(1, mesh.cmesh.tdim + 1):
#         volumes = mesh.cmesh.get_volumes(dim)
#         output('volumes of %d %dD entities:\nmin: %.7e mean: %.7e median:'
#                ' %.7e max: %.7e'
#                % (mesh.cmesh.num[dim], dim, volumes.min(), volumes.mean(),
#                   nm.median(volumes), volumes.max()))

#     euler = lambda mesh: nm.dot(mesh.cmesh.num, [1, -1, 1, -1])
#     ec = euler(mesh)
#     output('Euler characteristic:', ec)
# show_mesh_info("part1.vtk")